In [1]:
import pandas as pd
import plotly.express as px
import pathlib

import xlrd

In [32]:
ruta_git = pathlib.Path().cwd().parent.parent.parent

obs_0 = pd.read_excel(ruta_git/'data/raw/obs_1006.xlsx')
obs = pd.read_excel(ruta_git/'data/raw/obs_1306.xlsx')
obs.shape == obs_0.shape

True

In [33]:
obs = obs.dropna(how='all')
# parece que corrigieron cuando se arregló el campo Fecha
obs = obs.dropna(subset='Fecha')
#pero me quedan faltando 4
obs.tail(20)

KeyError: ['Fecha']

In [ ]:
obs.columns

In [ ]:
col_observaciones = ['Enviar', 'File', 'Nombre del observador(a)', 'Fecha de la observación', 'Nombre de docente', 'Número de cédula del docente', 'Asignatura en la que se hace la observación (implementa fichas)', 'Asignatura(s) que orienta regularmente el docente al grupo con el que dará la clase', 'Grado',
       'Número de ficha observada', 'Sesión', 'x_10', 'Internet Institucional.', 'N portatiles', 'N computadores de escritorio', 'N tablets', 'N celulares', 'N micro:bits',
       '¿Cuántos minutos transcurren entre el inicio de la clase estipulada y el inicio de la primera actividad?', '¿Se presentan los objetivos de aprendizaje de la lección?', '¿Se exploran los conocimientos previos de los estudiantes y su conexión con los temas de la lección?', '¿Se presentan y explican los conceptos claves que serán usados en la lección?', '¿Se preparó de forma previa el material requerido para la lección?', '¿Se gestionan correctamente los materiales e instrumentos para el desarrollo de la actividad?', '¿Cuántos estudiantes logran empezar la actividad sin requerir nuevas aclaraciones de las instrucciones?', '¿Cuántos estudiantes llevan a cabo la actividad desconectada según las instrucciones dadas?', '¿Cuántos estudiantes completan la actividad desconectada?',
       '¿Se invita a los estudiantes a compartir la solución que dieron a la actividad desconectada y reflexionar sobre otras posibles soluciones?', '¿Se hace un cierre formal de la actividad desconectada para destacar la relación entre ésta y el concepto de pensamiento computacional y/o programación de la clase?', 'x_28', 'x_29', '¿Cuántos estudiantes logran solucionar el reto de codificación propuesto con lo aprendido en la lección (Ej. el reto “Manos a la micro:bit” o “Para ir más lejos”)?', '¿Se usa el vocabulario adecuado para la enseñanza del pensamiento computacional (terminología correcta)?', '¿Se conectan los temas presentados con la vida diaria?', '¿Se presentaron problemas técnicos durante la clase?', '¿Estos problemas son resultado de falta de preparación?',
       '¿Se pudieron resolver oportunamente los problemas técnicos?', '¿Se siguen fielmente las actividades de la ficha?', '¿Se promueve la idea de que todos y todas pueden desarrollar su pensamiento computacional y habilidades de programación?', '¿Se valora el esfuerzo de los estudiantes para desarrollar las actividades propuestas?', '¿Se observa acompañamiento con estrategias de apoyo (aclaración de dudas, explicaciones, ejemplos adicionales, invitación a revisar partes específicas de las fichas, etc) a los estudiantes durante el desarrollo de las actividades?', 'x_40', '¿Se hace uso de la memoria colectiva?', '¿Se promueven los procesos de metacognición y reflexión?', 'Observaciones generales',
       'x_44', 'Nombre de la Institución Educativa', 'Otra sesión, ¿cuál?', 'Cantidad de estudiantes en el salón: Niños', 'Cantidad de estudiantes en el salón: Niñas', 'Se usan los dispositivos de computo',
       'Se usan las tarjetas micro:bit', 'Se cuenta con acceso a MakeCode', 'Se usa el acceso a internet', 'Se utilizan herramientas tecnológicas adicionales para apoyar el desarrollo de la sesión', 'Se usa otra herramienta. Cuál?', 'Momentos Usa que suceden en la sesión y la forma como se orientan', 'Momentos Modifica que suceden en la sesión y la forma como se orientan',
       'Momentos Crea que suceden en la sesión y la forma como se orientan', 'Se corrigen comentarios y comportamientos sexistas', ' Se estimula el liderazgo femenino', 'Se realizan acciones afirmativas en términos de género', 'Los retos de programación se enmarcan en una narrativa', 'Se dedica tiempo de la clase a hacer reflexiones sobre equidad de género, por ejemplo, aprovechando las historias que se encuentran en las fichas o reiterando normas de aula para evitar comentarios sexistas, entre otros.', 'observaciones',
       'IdEncuesta', 'Usuario', 'Fecha']

renombrar_columnas = dict(zip(obs.columns,col_observaciones))

renombrar = {}

renombrar['Sesión'] = {1:"Conectada",2:"Desconectada",
          3:"Ambas conectada y desconectada",
          4:"No sé"}

renombrar['¿Se presentan los objetivos de aprendizaje de la lección?'] = {1:"Sí, se presentan aprendizajes esperados relacionados con pensamiento computacional",
                  2:"Parcialmente. Se presentan objetivos, pero estos no están relacionados con pensamiento computacional. Ej. El objetivo presentado por el/la docente es que los estudiantes estén callados y trabajen juiciosos",
                  3:"No se presentan los objetivos que se espera lograr"}
renombrar['¿Se exploran los conocimientos previos de los estudiantes y su conexión con los temas de la lección?']={1:"Sí", 2:"Parcialmente. Se hace exploración de conocimientos previos, pero no se conectan claramente con el tema que se presenta",
          3:"No"}

renombrar["¿Se presentan y explican los conceptos claves que serán usados en la lección?"] = { 1:"Sí",
                                       2:"Parcialmente. Se hace una explicación, pero no es clara.", 3:"Parcialmente. Sólo se presenta, pero no se explica.",
                                        4:"No se hace una presentación, ni explicación de conceptos."}
renombrar["¿Se preparó de forma previa el material requerido para la lección?"] = {1:'Sí', 2:'Parcialmente', 3:"No", 4:"No aplica, no se requieren materiales para el desarrollo de las actividades"}

renombrar['¿Se gestionan correctamente los materiales e instrumentos para el desarrollo de la actividad?'] = {
    1:"Sí, se entregan los materiales requeridos de manera eficiente",
    2:"Parcialmente, se entregan los materiales requeridos pero el proceso es desordenado o toma mucho tiempo",
    3:"Parcialmente pues no se entregan materiales para el desarrollo de la actividad, pero se indica cómo diseñarlos o conseguirlos rápidamente",
    4:"No, no se entregan materiales ni se dan indicaciones sobre cómo gestionarlos",
    5:"No aplica, no se requieren materiales para el desarrollo de las actividades"
}
cuantos_est = {1:"Todo(a)s", 2:"La mayoría, pero no todo(a)s", 3:"Más de dos, pero menos de la mayoría",4:"Uno o dos"}
renombrar['¿Cuántos estudiantes logran empezar la actividad sin requerir nuevas aclaraciones de las instrucciones?']=cuantos_est
renombrar['¿Cuántos estudiantes llevan a cabo la actividad desconectada según las instrucciones dadas?'] = cuantos_est
renombrar['¿Cuántos estudiantes completan la actividad desconectada?'] = cuantos_est
renombrar['¿Se invita a los estudiantes a compartir la solución que dieron a la actividad desconectada y reflexionar sobre otras posibles soluciones?']={1:'Sí', 2:"Parcialmente, no se invita a los estudiantes a compartir verbalmente sus soluciones, pero sí se les invita a movilizarse y observar las soluciones planteadas por otros grupos", 3:"Parcialmente, se permite a los estudiantes participar, pero se destaca una única solución posible", 4:"No"}

renombrar['¿Se hace un cierre formal de la actividad desconectada para destacar la relación entre ésta y el concepto de pensamiento computacional y/o programación de la clase?'] = {1:"Sí, se hacen preguntas que permiten a los estudiantes conectar la actividad desarrollada con el concepto clave de la lección", 2:"Parcialmente, se indica la finalización de la actividad y el/la docente es quien explica la relación entre la actividad y el concepto clave de la lección", 3:"Parcialmente, se da cierre a la actividad, pero no se destaca su relación con el concepto clave de la lección", 4:"No se hace cierre formal de la actividad"}

renombrar['Momentos Usa que suceden en la sesión y la forma como se orientan'] = {'USA1':"Liderado y realizado por el/la docente, sin participación de los estudiantes", 'USA2':'Trabajo conjunto de Docente y Estudiantes', 'USA3':'Estudiantes desarrollan el trabajo de forma autónoma'}

renombrar['Momentos Modifica que suceden en la sesión y la forma como se orientan'] = {'MOD1':'Liderado y realizado por el/la docente, sin participación de los estudiantes', 'MOD2':'Trabajo conjunto de Docente y Estudiantes', 'MOD3':'Estudiantes desarrollan el trabajo de forma autónoma'}
renombrar['Momentos Crea que suceden en la sesión y la forma como se orientan']={'CRE1':"Liderado y realizado por el/la docente, sin participación de los estudiantes",'CRE2':'Trabajo conjunto de Docente y Estudiantes',"CRE3":"Estudiantes desarrollan el trabajo de forma autónoma"}
renombrar['¿Cuántos estudiantes logran solucionar el reto de codificación propuesto con lo aprendido en la lección (Ej. el reto “Manos a la micro:bit” o “Para ir más lejos”)?'] = cuantos_est|{5:"No aplica. No se presentó un reto de codificación"}

renombrar['¿Se usa el vocabulario adecuado para la enseñanza del pensamiento computacional (terminología correcta)?'] = {1:'Sí, durante toda la clase',2:"Parcialmente, durante la mayor parte de la clase",3:"Parcialmente, durante la mitad de la clase",4:"Parcialmente, durante la menor parte de la clase",5:"No"}

renombrar['¿Se conectan los temas presentados con la vida diaria?'] = {1:'Sí, se hace uso de analogías y/o ejemplos de la vida cotidiana para aclarar conceptos nuevos (ej. Bucles, variables, condicionales).',2:'Parcialmente, se hace uso de analogías y/o ejemplos que son apropiados para los conceptos, pero no se hace clara la conexión entre estos',3:'Parcialmente, se hace uso de analogías y/o ejemplos de la vida cotidiana, pero estos no son apropiados para ilustrar los conceptos nuevos',4:'No se hace uso de analogías ni de ejemplos, se centra en lo técnico únicamente'}

renombrar["¿Se siguen fielmente las actividades de la ficha?"]={1:"Sí, durante toda la clase"
    ,2:"Parcialmente, durante la mayor parte de la clase",3:"Parcialmente, durante la mitad de la clase",4:"Parcialmente, durante la menor parte de la clase",5:"No"}

renombrar['¿Se promueve la idea de que todos y todas pueden desarrollar su pensamiento computacional y habilidades de programación?'] = {1:"Sí, durante toda la clase",2:"Parcialmente, durante la mayor parte de la clase",3:"Parcialmente, durante la mitad de la clase",4:"Parcialmente, durante la menor parte de la clase",5:"No"}

renombrar['¿Se valora el esfuerzo de los estudiantes para desarrollar las actividades propuestas?'] = {1:"Sí, se valora el esfuerzo de todos los estudiantes, independientemente de si lograron completar los retos planteados de forma exitosa.",2:"Parcialmente, pues sólo se valoran los resultados de los estudiantes que completan exitosamente las tareas o retos planteados",3:"Parcialmente, pues sólo se valora el esfuerzo de quienes no lograron completar exitosamente la tarea o reto planteado",4:"No, no se reconoce ni el esfuerzo, ni el resultado exitoso del trabajo desarrollado por los estudiantes",5:"No, no se valora el esfuerzo sino únicamente el resultado"}

renombrar["¿Se observa acompañamiento con estrategias de apoyo (aclaración de dudas, explicaciones, ejemplos adicionales, invitación a revisar partes específicas de las fichas, etc) a los estudiantes durante el desarrollo de las actividades?"] ={1:"Sí, se verifica el trabajo el trabajo de los y las estudiantes y se acompaña con estrategias de apoyo",2:"Parcialmente, se verifica el trabajo de los y las estudiantes, pero se acompaña con estrategias de apoyo sólo a quienes lo solicitan",3:"Parcialmente, el acompañamiento se limita a hacer preguntas sobre el avance y cumplimiento de las tareas y no se brindan estrategias de apoyo",4:"No, no se hace ningún tipo de acompañamiento, ni se brinda apoyo a los estudiantes que tienen dificultades"}

renombrar['¿Se hace uso de la memoria colectiva?'] = {1:"Sí, se construye desde cero un gráfico tipo mapa mental con la ayuda de los estudiantes",2:"Sí, el docente presenta una estructura inicial de temas y construye el resto de la memoria colectiva con los aportes de los estudiantes",3:"Parcialmente, se hace sólo un listado de términos claves, con la ayuda de los estudiantes",4:"Parcialmente, el docente presenta una memoria colectiva previamente hecha sin apoyo de los estudiantes",5:"No, no se hace uso de esta estrategia"}

renombrar['¿Se promueven los procesos de metacognición y reflexión?'] = {1:"Sí, se pide a los estudiantes que reflexionen sobre lo aprendido y auto evalúen su alcance parcial o total de los objetivos",2:"Parcialmente, se hacen preguntas conceptuales, pero no se hacen preguntas para promover la reflexión ni la metacognición",3:"No se hace ningún tipo de preguntas ni de reflexión al cierre"}

In [ ]:
renombrar

In [ ]:
obs_r = obs.rename(columns=renombrar_columnas)
obs_r = obs_r.replace(renombrar)
obs_r = obs_r.replace("NOO","NO")
obs_r = obs_r.replace("--","0")
obs_r

In [ ]:
obs_r = obs_r[~obs_r['Usuario'].isin(['cristianavella','lorenamolina'])]
obs_r = obs_r[obs_r['Nombre del observador(a)'].str.len()>6]
obs_r = obs_r[~obs_r['Número de cédula del docente'].isna()]
obs_r = obs_r[obs_r['Número de cédula del docente'] > 10000000]
obs_r = obs_r[~obs_r['Nombre de docente'].isin(["PEPITO PEREZ"])]
obs_r = obs_r.filter(regex="^(?!.*x_).")
obs_r

In [34]:
obs_r.to_excel("Observaciones_1306.xlsx")

NameError: name 'obs_r' is not defined

In [35]:
obs_r['Nombre del observador(a)'].unique()

NameError: name 'obs_r' is not defined

# Instantaneas

In [83]:
instantaneas = pd.read_excel(ruta_git/'data/raw/inst_1006.xlsx')

In [85]:
instantaneas.columns

Index(['id', 'inst_1', 'inst_2', 'inst_3', 'inst_4', 'inst_5', 'inst_6',
       'inst_7', 'inst_8', 'inst_9', 'inst_10', 'inst_11', 'inst_12',
       'inst_13', 'inst_14', 'inst_15', 'inst_16', 'inst_17', 'inst_18',
       'inst_19', 'inst_20', 'inst_21', 'inst_22', 'IdEncuesta', 'Usuario',
       'Fecha'],
      dtype='object')

In [ ]:
inst_cols = ['id', 'inst_1', 'inst_2', 'inst_3', 'inst_4', 'inst_5', 'inst_6',
       'inst_7', 'inst_8', 'inst_9', 'inst_10', 'inst_11', 'inst_12',
       'inst_13', 'inst_14', 'inst_15', 'inst_16', 'inst_17', 'inst_18',
       'inst_19', 'inst_20', 'inst_21', 'inst_22', 'IdEncuesta', 'Usuario',
       'Fecha']

## Gráfica Jose

In [8]:
datos = pd.read_excel('grafica1.xlsx', sheet_name='grafica')
datos

,Ficha,Presenta Ob,Conocimientos previos,Conceptos claves,Preparación de material,Gestión de materiales,Comparte solución,Cierre formal,Lenguaje técnico,Conexión vida diaria,Metacognición
0,Ficha 1,Si,Si,Si,Si,Si,Si,Parcialmente,Si,Parcialmente,Si
1,Sin respuesta,Si,Si,Si,Si,Si,No,Parcialmente,Parcialmente,No,Parcialmente
2,Ficha 3,Si,Si,Si,Si,Si,Si,No,Si,Si,Si
3,Ficha 1,Si,Si,Si,Si,Si,Si,Parcialmente,Parcialmente,Parcialmente,No
4,Ficha 2,Si,Si,Si,Si,Si,No,No,Si,No,Parcialmente
5,Sin respuesta,Si,Si,Si,Si,Si,Si,Si,Parcialmente,Si,Si
6,Sin respuesta,No,No,Parcialmente,Si,Si,No,No,Parcialmente,No,No
7,Ficha 2,Si,Si,Si,Si,Si,Si,Parcialmente,Si,Si,Parcialmente
8,Ficha 1,Si,Si,Si,Si,Si,Parcialmente,Parcialmente,Si,Parcialmente,No
9,Ficha 2,Si,Si,Si,Si,Si,No,No,Si,Si,Si


In [9]:
datos.columns

Index(['Ficha', 'Presenta Ob', 'Conocimientos previos', 'Conceptos claves',
       'Preparación de material', 'Gestión de materiales', 'Comparte solución',
       'Cierre formal', 'Lenguaje técnico', 'Conexión vida diaria',
       'Metacognición'],
      dtype='object')

In [10]:
dat = datos.melt(id_vars='Ficha', value_name='Implementación', var_name='Concepto')
dat = dat.reset_index()

In [11]:
datp = dat.pivot_table(index=['Ficha','Concepto', 'Implementación'], values='index', aggfunc='count').reset_index()
total = dat.pivot_table(index=['Ficha','Concepto'], values='index', aggfunc='count').reset_index().rename(columns={'index':'Total'})

datp = datp.merge(total)
datp['Frecuencia'] = datp['index']/datp['Total']
datp.to_feather('graficajose.feather')

In [15]:
list(datp.Ficha.unique())

['Ficha 1', 'Ficha 2', 'Ficha 3', 'Ficha 4', 'Sin respuesta']

In [12]:
fig1= px.bar(datp, x='Concepto',y='Frecuencia', color='Implementación', facet_col='Ficha',  color_discrete_sequence=px.colors.qualitative.Pastel,
                 template="plotly_white", category_orders={'Implementación':['Si','Parcialmente','No'],
                                                           'Concepto' : ['Presenta Ob', 'Conocimientos previos', 'Conceptos claves',
       'Preparación de material', 'Gestión de materiales', 'Comparte solución',
       'Cierre formal', 'Lenguaje técnico', 'Conexión vida diaria',
       'Metacognición']})
fig1

In [13]:
fig2 = px.bar(datp, y='Concepto',x='Frecuencia', color='Implementación', facet_row='Ficha', orientation='h', color_discrete_sequence=px.colors.qualitative.Pastel,
                 template="plotly_white", category_orders={'Implementación':['Si','Parcialmente','No'],
                                                           'Concepto' : ['Presenta Ob', 'Conocimientos previos', 'Conceptos claves',
       'Preparación de material', 'Gestión de materiales', 'Comparte solución',
       'Cierre formal', 'Lenguaje técnico', 'Conexión vida diaria',
       'Metacognición']}, height=1500, width=700)

fig2